In [13]:
# %matplotlib Qt
%matplotlib inline

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import jdatetime

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV  # Linear least squares with l2 regularization
from sklearn.metrics import mean_absolute_error, mean_squared_error

from modules import LLS

In [15]:
data = pd.read_csv('data/Dollar_Rial_Price_Dataset.csv')
data.head()

,Unnamed: 0,Date,Persian_Date,Open,Low,High,Close
0,0,11/27/2011,1390/09/06,"13,700","13,700","13,700","13,700"
1,1,11/28/2011,1390/09/07,"13,440","13,440","13,440","13,440"
2,2,11/29/2011,1390/09/08,"13,350","13,350","13,350","13,350"
3,3,11/30/2011,1390/09/09,"13,400","13,400","13,400","13,400"
4,4,12/1/2011,1390/09/10,"13,500","13,500","13,500","13,500"


### Pre Proccessing

In [16]:
def str_to_int(x):
    return int(x.replace(",", ""))

def str_to_jdate(persian_date):
    parts = persian_date.split('/')
    return jdatetime.date(int(parts[0]), int(parts[1]), int(parts[2]))

data["Persian_Date"] = data["Persian_Date"].apply(str_to_jdate)
data["High"] = data["High"].apply(str_to_int)
data["Low"] = data["Low"].apply(str_to_int)
data["Open"] = data["Open"].apply(str_to_int)
data["Close"] = data["Close"].apply(str_to_int)
data.head()

,Unnamed: 0,Date,Persian_Date,Open,Low,High,Close
0,0,11/27/2011,1390-09-06,13700,13700,13700,13700
1,1,11/28/2011,1390-09-07,13440,13440,13440,13440
2,2,11/29/2011,1390-09-08,13350,13350,13350,13350
3,3,11/30/2011,1390-09-09,13400,13400,13400,13400
4,4,12/1/2011,1390-09-10,13500,13500,13500,13500


In [17]:
ahmadinejad = data.loc[data["Persian_Date"] < jdatetime.date(1392, 5, 12)].reset_index()
rohani = data.loc[(data["Persian_Date"] >= jdatetime.date(1392, 5, 12)) & (data["Persian_Date"] < jdatetime.date(1400, 5, 12))].reset_index()
raisi = data.loc[(data["Persian_Date"] >= jdatetime.date(1400, 5, 12))].reset_index()

# ahmadinejad
# rohani
raisi

,index,Unnamed: 0,Date,Persian_Date,Open,Low,High,Close
0,2857,2857,8/3/2021,1400-05-12,255790,255690,258140,257290
1,2858,2858,8/4/2021,1400-05-13,257250,254690,257340,256190
2,2859,2859,8/5/2021,1400-05-14,256490,255790,256740,256190
3,2860,2860,8/7/2021,1400-05-16,254190,253990,257140,256380
4,2861,2861,8/8/2021,1400-05-17,256290,252390,257440,256890
...,...,...,...,...,...,...,...,...
448,3305,3305,7/1/2023,1402-04-10,492660,492600,498400,498220
449,3306,3306,7/2/2023,1402-04-11,500810,498600,500900,499340
450,3307,3307,7/3/2023,1402-04-12,503210,501400,503400,502000
451,3308,3308,7/4/2023,1402-04-13,502140,498300,502200,500750


In [26]:
def calculate_growth_rate(highest_price, lowest_price):
    return f"%{round(((highest_price - lowest_price) / lowest_price) * 100, 0)}"

In [27]:
ahmadinejad_high = ahmadinejad['High'].max()
ahmadinejad_low = ahmadinejad['High'].min()
ahmadinejad_growth = calculate_growth_rate(ahmadinejad_high, ahmadinejad_low)

rohani_high = rohani['High'].max()
rohani_low = rohani['High'].min()
rohani_growth = calculate_growth_rate(rohani_high, rohani_low)

raisi_high = raisi['High'].max()
raisi_low = raisi['High'].min()
raisi_growth = calculate_growth_rate(raisi_high, raisi_low)

In [28]:
# Calculation of the highest and lowest price in each period
results = pd.DataFrame({
    "President": ["Ahmadinejad", "Rohani", "Raisi"],
    "Highest Price": [ahmadinejad_high, rohani_high, raisi_high],
    "Lowest Price": [ahmadinejad_low, rohani_low, raisi_low],
    "Growth Rate (%)": [ahmadinejad_growth, rohani_growth, raisi_growth]
})
results

,President,Highest Price,Lowest Price,Growth Rate (%)
0,Ahmadinejad,39700,13350,%197.0
1,Rohani,320060,29150,%998.0
2,Raisi,555600,254300,%118.0
